## Задание 1.
- Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
- Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
from pymongo import MongoClient

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
URL_HH_PREFIX = 'https://hh.ru/search/vacancy?text='
URL_HH_SUFFIX = '&from=suggest_post&area=1'

In [3]:
def get_soups_from_url(url):
    response = requests.get(url, headers = {'User-Agent': USER_AGENT})
    soups = []
    if response.status_code == 200:
        soups.append(bs(response.content, 'html.parser'))
        while soups[-1].find_all('a', {'data-qa': 'pager-next'}):
            sleep(5)
            response = requests.get(url = url_hh + '&page=' + str(len(soups)), headers = {'User-Agent': USER_AGENT})
            if response.status_code == 200:
                soups.append(bs(response.content, 'html.parser'))
            else:
                print(f'Парсинг завершен досрочно. Удалось спарсить только страниц: {len(soups)}')
                break
        else:
            print(f'Парсинг успешно завершен. Удалось спарсить страниц: {len(soups)}')
    else:
        print(f'Парсинг не выполнен. Проверьте URL или другие параметры')
    return soups

In [11]:
def get_vacancies_from_soups_hh(url):
    vacancies = []
    for soup in soups:
        for vacancy in soup.find_all('div', {'class': 'serp-item'}):
            vacancy_name = vacancy.find('a', {'class': 'serp-item__title'}).text
            vacancy_company = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).text
            vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            salary_min, salary_max, salary_currency = '', '', ''
            if vacancy_salary:
                vacancy_salary = vacancy_salary.text
                salary_currency = vacancy_salary[vacancy_salary.rfind(' ')+1:].upper().replace('.','')
                vacancy_salary = vacancy_salary[:vacancy_salary.rfind(' ')]
                if vacancy_salary.find('–') > 0:
                    salary_min = vacancy_salary[:vacancy_salary.find('–')].strip().replace(' ','')
                    salary_max = vacancy_salary[vacancy_salary.find('–')+1:].strip().replace(' ','')
                elif vacancy_salary.find('от ') == 0:
                    salary_min = vacancy_salary[3:].strip().replace(' ','')
                elif vacancy_salary.find('до ') == 0:
                    salary_max = vacancy_salary[3:].strip().replace(' ','')
            vacancy_link = vacancy.find('a')['href']
            vacancy_source = 'HeadHunter'
            vacancies.append({'vacancy_name': vacancy_name,
                              'vacancy_company': vacancy_company,
                              'salary_min': salary_min,
                              'salary_max': salary_max,
                              'salary_currency': salary_currency,
                              'vacancy_link': vacancy_link,
                              'vacancy_source': vacancy_source})
    return vacancies

In [12]:
text = input('Введите профессию, должность или компанию: ')
url_hh = URL_HH_PREFIX + text.strip().replace(' ','+') + URL_HH_SUFFIX
soups = get_soups_from_url(url_hh)
vacancies_hh = get_vacancies_from_soups_hh(soups)

Введите профессию, должность или компанию: Data Science
Парсинг успешно завершен. Удалось спарсить страниц: 9


In [13]:
client = MongoClient()

In [14]:
db = client.vacancies_hh

In [15]:
collection_vacancies_hh = db.hh

In [20]:
def cheak_and_save_vacancies_in_db(vacancies):
    for vacancy in vacancies:
        if not len(list(collection_vacancies_hh.find({'vacancy_link': vacancy['vacancy_link']}))):
            collection_vacancies_hh.insert_one(vacancy)

In [21]:
cheak_and_save_vacancies_in_db(vacancies_hh)

In [22]:
result_find = list(collection_vacancies_hh.find())
len(result_find)

180

In [23]:
def show_vacancies(salary):
    print([collection_vacancies_hh.])

show_vacancies(200000)

TypeError: '>' not supported between instances of 'str' and 'int'